In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import wandb

import torch
from torch import nn
from torchvision import models
import torch.optim as optim

from dataset import get_dataloaders, get_datasets
from utils import seed_everything
from trainer import Trainer

# Params
Image.MAX_IMAGE_PIXELS = 1e11
CFG = {
    'seed': 42,
    'cv_fold': 5,
    'base_model': 'efficientnet_b1',   # resnet18/34/50, efficientnet_b0/b1/b2/b3/b4, convnext_tiny, vit_b_16, swin_t
    'img_size': 1024,
    'batch_size': 16,
    'freeze_epochs': 1,
    'epochs': 5,
    'base_lr': 1e-3,
    'affine_degrees': 20,
    'affine_translate': (0.1, 0.1),
    'affine_scale': (1.0, 1.4),
    'dataloader_num_workers': 8,
    'scheduler_step_size': 2,
    'img_color_mean': [0.8708488980328596, 0.75677901508938, 0.8545134911215124],
    'img_color_std': [0.08086288591996027, 0.11553960008706814, 0.06914169213328555],
    'optimizer': 'AdamW',
    'scheduler': 'CosineAnnealingLR',
    'lr_gamma': 0.1,
    'lr_cycl_step_size': 3,
    'sgd_momentum': 0.9,
    'valid_type': 'all_tmas',
    # 'hed_theta': 0.05,
    'color_jitter': {'brightness': 0.2, 'contrast': 0.2, 'saturation': 0.2, 'hue': 0.2},
    'gaussian_blur_kernel_size': 10
}
tags = ['torch', 'thumbnails', 'cv']
notes = 'randomresizedcrop + 20deg aug + colorjitter'
plot_samples = False

# Wandb
wandb.login(key='1b0401db7513303bdea77fb070097f9d2850cf3b')
run = wandb.init(project='kaggle-ubc-ocean', config=CFG, tags=tags, notes=notes)

# Label encoder/decoder
encode = {'HGSC': 0, 'LGSC': 1, 'EC': 2, 'CC': 3, 'MC': 4}
decode = {v: k for k, v in encode.items()}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Paths
root = '/media/latlab/MR/projects/kaggle-ubc-ocean'
data_dir = os.path.join(root, 'data')
results_dir = os.path.join(root, 'results')
train_csv = 'train.csv'
train_image_dir = os.path.join(data_dir, 'train_images')
train_thumbnail_dir = os.path.join(data_dir, 'train_thumbnails')

# Seed
seed_everything(CFG['seed'])

# Load data
df = pd.read_csv(os.path.join(data_dir, train_csv))
df['label'] = df.loc[:,'label'].map(encode)

# Functions
def train_model(CFG, train_image_dir, train_thumbnail_dir, df_train, df_validation, encode, wandb_log=False):
    # Data loaders
    datasets = get_datasets(CFG, train_image_dir, train_thumbnail_dir, df_train, df_validation)
    dataloaders = get_dataloaders(CFG, datasets)

    # Model definition
    model = models.get_model(CFG['base_model'], weights='DEFAULT').to(device)

    # Freeze all parameters
    for param in model.parameters():
        param.requires_grad = False
    
    # Replace the last fully-connected layer
    if CFG['base_model'].startswith('resnet'):
        model.fc = nn.Linear(in_features=model.fc.in_features, out_features=len(encode)).to(device)
    elif CFG['base_model'].startswith('efficientnet'):
        model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=len(encode), bias=True).to(device)
    elif CFG['base_model'].startswith('convnext'):
        model.classifier[2] = nn.Linear(in_features=model.classifier[2].in_features, out_features=len(encode), bias=True).to(device)
    elif CFG['base_model'].startswith('vit'):
        model.heads.head = nn.Linear(in_features=model.heads.head.in_features, out_features=len(encode), bias=True).to(device)
    elif CFG['base_model'].startswith('swin'):
        model.head = nn.Linear(in_features=model.head.in_features, out_features=len(encode), bias=True).to(device)
    
    # Loss function
    loss_fn = nn.CrossEntropyLoss()

    # Optimizer
    if CFG['optimizer'] == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=CFG['base_lr'], momentum=CFG['sgd_momentum'])
    elif CFG['optimizer'] == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=CFG['base_lr'])
    elif CFG['optimizer'] == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=CFG['base_lr'])
    
    # Scheduler
    if CFG['scheduler'] == 'StepLR':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=CFG['scheduler_step_size'], gamma=CFG['lr_gamma'], verbose=True)
    elif CFG['scheduler'] == 'CyclicLR':
        scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=CFG['base_lr'], max_lr=CFG['base_lr']*10,
                                                step_size_up=3, cycle_momentum=False, mode='triangular2', verbose=True)
    elif CFG['scheduler'] == 'CosineAnnealingLR':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG['epochs']+CFG['freeze_epochs'], verbose=True)
    elif CFG['scheduler'] == 'OneCycleLR':
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=CFG['base_lr'], total_steps=CFG['epochs']+CFG['freeze_epochs'], verbose=True)

    # Training
    trainer = Trainer(model, dataloaders, loss_fn, optimizer, scheduler, device, metric='balanced_accuracy', wandb_log=wandb_log)
    model, _ = trainer.train_epochs(num_epochs=CFG['freeze_epochs'])
    trainer.unfreeze()
    model, balanced_acc = trainer.train_epochs(num_epochs=CFG['epochs'])
    return model, balanced_acc

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: naraiadam88. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/latlab/.netrc


In [2]:
# Show training data
if plot_samples:
    dataloaders = get_dataloaders(CFG, get_datasets(CFG, train_image_dir, train_thumbnail_dir, df, df))
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloaders['train']):
            plt.figure(figsize=(16, 12))
            for i in range(len(X)):
                plt.subplot(int(np.ceil(len(X)/6)), 6, i+1)
                img_data = X[i].permute(1, 2, 0).cpu().numpy()
                plt.imshow(img_data)
                plt.title(f'{decode[y[i].item()]}')
            if batch >= 1:
                break

In [3]:
skf = StratifiedKFold(n_splits=CFG['cv_fold'], random_state=CFG['seed'], shuffle=True)
balanced_acc_list = []
if CFG['valid_type'] == 'all_tmas':
    df_tma = df[df['is_tma']==True].copy().reset_index(drop=True)
    df_wsi = df[df['is_tma']==False].copy().reset_index(drop=True)
    for cv, (train_index, valid_index) in enumerate(skf.split(X=np.zeros(len(df_wsi['label'])), y=df_wsi['label'])):
        print(f"Cross-validation fold {cv+1}/{CFG['cv_fold']}")
        df_train = df_wsi.iloc[train_index]
        df_validation = df_wsi.iloc[valid_index]
        df_validation = pd.concat([df_validation, df_tma], ignore_index=True).sample(frac=1).reset_index(drop=True)
        run_name = f'{run.name}-cv{cv+1}'
        model, balanced_acc = train_model(CFG, train_image_dir, train_thumbnail_dir, df_train, df_validation, encode)
        balanced_acc_list.append(balanced_acc)
        torch.save(model.state_dict(), os.path.join(results_dir, 'models', f'ubc-ocean-{run_name}.pt'))
        wandb.log({f'balanced_acc_cv{cv+1}': balanced_acc})
elif CFG['valid_type'] == 'standard':
    for cv, (train_index, valid_index) in enumerate(skf.split(X=np.zeros(len(df['label'])), y=df['label'])):
        print(f"Cross-validation fold {cv+1}/{CFG['cv_fold']}")
        df_train = df.iloc[train_index]
        df_validation = df.iloc[valid_index]
        run_name = f'{run.name}-cv{cv+1}'
        model, balanced_acc = train_model(CFG, train_image_dir, train_thumbnail_dir, df_train, df_validation, encode)
        balanced_acc_list.append(balanced_acc)
        torch.save(model.state_dict(), os.path.join(results_dir, 'models', f'ubc-ocean-{run_name}.pt'))
        wandb.log({f'balanced_acc_cv{cv+1}': balanced_acc})
wandb.log({f'mean_balanced_acc': np.mean(balanced_acc_list)})
wandb.finish()

Cross-validation fold 1/5
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/1
----------


100%|██████████| 8/8 [00:11<00:00,  1.46s/it]


Adjusting learning rate of group 0 to 9.3301e-04.
train loss: 1.5094, test loss: 1.4552, balanced_accuracy: 0.3838

Training complete in 0m 50s
Final balanced_accuracy: 0.383839

Epoch 1/5
----------


100%|██████████| 8/8 [00:11<00:00,  1.47s/it]


Adjusting learning rate of group 0 to 7.5000e-04.
train loss: 1.2320, test loss: 1.1959, balanced_accuracy: 0.5341

Epoch 2/5
----------


...

Adjusting learning rate of group 0 to 6.6987e-05.
train loss: 0.6305, test loss: 1.0575, balanced_accuracy: 0.5899

Epoch 5/5
----------


100%|██████████| 8/8 [00:14<00:00,  1.87s/it]

Adjusting learning rate of group 0 to 0.0000e+00.
train loss: 0.6383, test loss: 1.1004, balanced_accuracy: 0.6015

Training complete in 4m 28s
Final balanced_accuracy: 0.601463



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


balanced_acc_cv1,▁
balanced_acc_cv2,▁
balanced_acc_cv3,▁
balanced_acc_cv4,▁
balanced_acc_cv5,▁
mean_balanced_acc,▁
balanced_acc_cv1,0.5756
balanced_acc_cv2,0.68426
balanced_acc_cv3,0.6636
balanced_acc_cv4,0.59879
balanced_acc_cv5,0.60146


In [4]:
# Final training on all data
model, _ = train_model(CFG, train_image_dir, train_thumbnail_dir, df, df, encode, wandb_log=False)
torch.save(model.state_dict(), os.path.join(results_dir, 'models', f'ubc-ocean-{run.name}.pt'))

Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/1
----------


100%|██████████| 34/34 [00:46<00:00,  1.38s/it]


Adjusting learning rate of group 0 to 9.3301e-04.
train loss: 1.4989, test loss: 1.3733, balanced_accuracy: 0.4155

Training complete in 1m 36s
Final balanced_accuracy: 0.415504

Epoch 1/5
----------


100%|██████████| 34/34 [00:46<00:00,  1.38s/it]


Adjusting learning rate of group 0 to 7.5000e-04.
train loss: 1.2546, test loss: 0.8609, balanced_accuracy: 0.6130

Epoch 2/5
----------


...

Adjusting learning rate of group 0 to 2.5000e-04.
train loss: 0.8247, test loss: 0.5796, balanced_accuracy: 0.7603

Epoch 4/5
----------


100%|██████████| 34/34 [00:47<00:00,  1.39s/it]


Adjusting learning rate of group 0 to 6.6987e-05.
train loss: 0.7038, test loss: 0.4588, balanced_accuracy: 0.7931

Epoch 5/5
----------


100%|██████████| 34/34 [00:47<00:00,  1.39s/it]


Adjusting learning rate of group 0 to 0.0000e+00.
train loss: 0.6430, test loss: 0.4335, balanced_accuracy: 0.8073

Training complete in 8m 5s
Final balanced_accuracy: 0.807319

